In [18]:
import torch
import torch.nn as nn
from torch.nn import functional as F

torch.manual_seed(147)

#@ Setting up hyper-parameters:
batch_size=16 #how many independent sequence will we process in parallel
block_size=32 #maximum context length for prediction.
max_iter=5000 #for training
eval_interval=100
learning_rate=1e-3
device='cuda' if torch.cuda.is_available() else 'cpu'
eval_iters=200
n_embd=64
n_head=4
n_layer=4
dropout=0.0

In [2]:
#@ Dataset for training gpt, we are using is tiny shakespares dataset:

!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2025-02-13 14:51:11--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.009s  

2025-02-13 14:51:11 (119 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [19]:
#@ Reading the file:
with open('input.txt', 'r', encoding='utf-8') as f:
  text=f.read()

In [4]:
print('length of dataset in characters:', len(text))

length of dataset in characters: 1115394


In [5]:
#@ inspecting:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [6]:
#@ Unique characters for the code:

chars=sorted(list(set(text)))
# print(chars)
vocab_size=len(chars)
# print(vocab_size) # unique characters len
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [7]:
#@ mapping from characters to integer:
str_int={ch:i for i, ch in enumerate(chars)}
int_str={i:ch for i, ch in enumerate(chars)}

encode=lambda s:[str_int[c] for c in s]
decode=lambda l: ''.join([int_str[i] for i in l])

print(encode('hello, i am prijal!'))
print(decode(encode('hello, i am prijal!')))

[46, 43, 50, 50, 53, 6, 1, 47, 1, 39, 51, 1, 54, 56, 47, 48, 39, 50, 2]
hello, i am prijal!


In [14]:
#@ Encoding the entire dataset:
import torch
data=torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
# print(data[:1000])

torch.Size([1115394]) torch.int64


In [9]:
#@ Train-test split:
n=int(0.9*len(data)) # first 90% will be training
train_data=data[:n]
val_data=data[n:]

In [17]:
#@ constructing batch:
def get_batch(split):
  data=train_data if split == 'train' else val_data
  ix=torch.randint(len(data)-block_size, (batch_size,))
  x=torch.stack([data[i:i+block_size] for i in ix])
  y=torch.stack([data[i+1:i+block_size+1] for i in ix])
  x, y=x.to(device), y.to(device)
  return x, y

In [ ]:
#@ for loss calculation:
@torch.no_grad()
def estimate_loss():
  output={}
  model.eval()
  for split in ['train', 'test']:
    losses=torch.zeros(eval_iters)
    for k in range(eval_iters):
      X, Y=get_batch(split)
      logits, loss=model(X, Y)
      losses[k]=loss.item()
    output[split]=losses.mean()
  model.train()
  return output

In [ ]:
class Head(nn.Module):
  '''One head of self-attention'''
  def __init__(self, head_size):
    super().__init__()
    self.query=nn.Linear(n_embd, head_size, bias=False)
    self.key=nn.Linear(n_embd, head_size, bias=False)
    self.value=nn.Linear(n_embd, head_size, bias=False)

    self.dropout=nn.Dropout(dropout)

In [13]:
class BigramLanguageModel(nn.Module):
  def __init__(self, vocab_size):
    super().__init__()
    self.token_embedding_table=nn.Embedding(vocab_size, n_embd)

  def forward(self, idx, targets=None):
    logits=self.token_embedding_table(idx) #(B,T,C)

    if targets is None:
      loss=None

    else:
      B, T, C =logits.shape
      logits=logits.view(B*T, C)
      targets=targets.view(B*T)
      loss=F.cross_entropy(logits, targets)

    return logits, loss

  def generate(self, idx, max_new_tokens):
    #idx id (B, T) array of indices in the current context
    for _ in range(max_new_tokens):
      #getting the prediction:
      logits, loss=self(idx)

      #geting only the last time step:
      logits=logits[:, -1, :] #(B, C)

      #softmax:
      probs=F.softmax(logits, dim=-1)

      #sample from distribution:
      idx_next=torch.multinomial(probs, num_samples=1) #(B, 1)

      #append
      idx=torch.cat((idx, idx_next), dim=1) #(B, T+1)

    return idx

model=BigramLanguageModel(vocab_size)
logits, loss=m(xb, yb)

print(decode(m.generate(idx=torch.zeros((1, 1), dtype=torch.long),
                        max_new_tokens=100)[0].tolist()))



bnkvUpoZfSSPBq',grXmP3rQLAjJVwI-abnVdHhK''nW.yNKgzhSTPKCI:zpvWbNGfgNesjfNL,f.I:oMiknvy3:G GpD,tcY;

